## 1. Inspecting the data
<p><img src="https://assets.datacamp.com/production/project_1416/img/schoolbus.jpg" alt="New York City schoolbus" height="300px" width="300px"></p>
<p>Photo by <a href="https://unsplash.com/@jannis_lucas">Jannis Lucas</a> on <a href="https://unsplash.com">Unsplash</a>.
<br></p>
<p>Every year, American high school students take SATs, which are standardized tests intended to measure literacy, numeracy, and writing skills. There are three sections - reading, math, and writing, each with a maximum score of 800 points. These tests are extremely important for students and colleges, as they play a pivotal role in the admissions process.</p>
<p>Analyzing the performance of schools is important for a variety of stakeholders, including policy and education professionals, researchers, government, and even parents considering which school their children should attend. </p>
<p>In this notebook, we will take a look at data on SATs across public schools in New York City. Our database contains a single table:</p>
<h1 id="schools"><code>schools</code></h1>
<table>
<thead>
<tr>
<th>column</th>
<th>type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>school_name</code></td>
<td><code>varchar</code></td>
<td>Name of school</td>
</tr>
<tr>
<td><code>borough</code></td>
<td><code>varchar</code></td>
<td>Borough that the school is located in</td>
</tr>
<tr>
<td><code>building_code</code></td>
<td><code>varchar</code></td>
<td>Code for the building</td>
</tr>
<tr>
<td><code>average_math</code></td>
<td><code>int</code></td>
<td>Average math score for SATs</td>
</tr>
<tr>
<td><code>average_reading</code></td>
<td><code>int</code></td>
<td>Average reading score for SATs</td>
</tr>
<tr>
<td><code>average_writing</code></td>
<td><code>int</code></td>
<td>Average writing score for SATs</td>
</tr>
<tr>
<td><code>percent_tested</code></td>
<td><code>numeric</code></td>
<td>Percentage of students completing SATs</td>
</tr>
</tbody>
</table>
<p>Let's familiarize ourselves with the data by taking a looking at the first few schools!</p>

In [2]:
# !pip install pandas
# !pip install ipython-sql

import pandas as pd
import sqlite3

dt = pd.read_csv("datasets/schools_modified.csv")
cnn = sqlite3.connect('createdb.db')
dt.to_sql("schools", cnn)
%load_ext sql
%sql sqlite:///createdb.db



ValueError: Table 'schools' already exists.

In [1]:
%%sql 
    
-- Select all columns from the database
SELECT *
FROM schools
-- Display only the first ten rows
LIMIT 10;

UsageError: Cell magic `%%sql` not found.


## 2. Finding missing values
<p>It looks like the first school in our database had no data in the <code>percent_tested</code> column! </p>
<p>Let's identify how many schools have missing data for this column, indicating schools that did not report the percentage of students tested. </p>
<p>To understand whether this missing data problem is widespread in New York, we will also calculate the total number of schools in the database.</p>

In [27]:
%%sql

-- Count rows with percent_tested missing and total number of schools
SELECT 
    COUNT(school_name) - COUNT(percent_tested) AS percent_tested_missing,
    COUNT(school_name) AS num_schools
FROM schools

    

 * sqlite:///createdb.db
Done.


percent_tested_missing,num_schools
20,375


## 3. Schools by building code
<p>There are 20 schools with missing data for <code>percent_tested</code>, which only makes up 5% of all rows in the database.</p>
<p>Now let's turn our attention to how many schools there are. When we displayed the first ten rows of the database, several had the same value in the <code>building_code</code> column, suggesting there are multiple schools based in the same location. Let's find out how many unique school locations exist in our database. </p>

In [29]:
%%sql
SELECT 
-- Count the number of unique building_code values
    COUNT(DISTINCT(building_code)) AS num_building_code
FROM schools    



 * sqlite:///createdb.db
Done.


num_building_code
233


## 4. Best schools for math
<p>Out of 375 schools, only 233 (62%) have a unique <code>building_code</code>! </p>
<p>Now let's start our analysis of school performance. As each school reports individually, we will treat them this way rather than grouping them by <code>building_code</code>. </p>
<p>First, let's find all schools with an average math score of at least 80% (out of 800). </p>

In [30]:
%%sql

-- Select school and average_math
SELECT 
    school_name,
    average_math
FROM schools
-- Filter for average_math 640 or higher
WHERE average_math >= 640
-- Display from largest to smallest average_math
ORDER BY average_math DESC;

 * sqlite:///createdb.db
Done.


school_name,average_math
Stuyvesant High School,754
Bronx High School of Science,714
Staten Island Technical High School,711
Queens High School for the Sciences at York College,701
"High School for Mathematics, Science, and Engineering at City College",683
Brooklyn Technical High School,682
Townsend Harris High School,680
High School of American Studies at Lehman College,669
"New Explorations into Science, Technology and Math High School",657
Eleanor Roosevelt High School,641


## 5. Lowest reading score
<p>Wow, there are only ten public schools in New York City with an average math score of at least 640!</p>
<p>Now let's look at the other end of the spectrum and find the single lowest score for reading. We will only select the score, not the school, to avoid naming and shaming!</p>

In [33]:
%%sql

-- Find lowest average_reading
SELECT 
    school_name,
    average_reading
FROM schools
ORDER BY average_reading ASC
LIMIT 1; 



 * sqlite:///createdb.db
Done.


school_name,average_reading
International High School at Prospect Heights,302


## 6. Best writing school
<p>The lowest average score for reading across schools in New York City is less than 40% of the total available points! </p>
<p>Now let's find the school with the highest average writing score.</p>

In [35]:
%%sql

-- Find the top score for average_writing
SELECT 
    school_name,
    MAX(average_writing) AS max_writing
FROM schools
-- Group the results by school
GROUP BY school_name

-- Sort by max_writing in descending order
ORDER BY max_writing DESC
-- Reduce output to one school
LIMIT 1;

 * sqlite:///createdb.db
Done.


school_name,max_writing
Stuyvesant High School,693


## 7. Top 10 schools
<p>An average writing score of 693 is pretty impressive! </p>
<p>This top writing score was at the same school that got the top math score, Stuyvesant High School. Stuyvesant is widely known as a perennial top school in New York. </p>
<p>What other schools are also excellent across the board? Let's look at scores across reading, writing, and math to find out.</p>

In [36]:
%%sql

-- Calculate average_sat
SELECT 
    school_name,
    average_reading + average_writing + average_math AS average_sat
FROM schools
-- Group by school_name
GROUP BY school_name
-- Sort by average_sat in descending order
ORDER BY average_sat DESC
-- Display the top ten results
LIMIT 10;

 * sqlite:///createdb.db
Done.


school_name,average_sat
Stuyvesant High School,2144
Staten Island Technical High School,2041
Bronx High School of Science,2041
High School of American Studies at Lehman College,2013
Townsend Harris High School,1981
Queens High School for the Sciences at York College,1947
Bard High School Early College,1914
Brooklyn Technical High School,1896
"High School for Mathematics, Science, and Engineering at City College",1889
Eleanor Roosevelt High School,1889


## 8. Ranking boroughs
<p>There are four schools with average SAT scores of over 2000! Now let's analyze performance by New York City borough. </p>
<p>We will build a query that calculates the number of schools and the average SAT score per borough!</p>

In [38]:
%%sql

-- Select borough and a count of all schools, aliased as num_schools
SELECT 
    borough,
    COUNT(school_name) AS num_schools,
-- Calculate the sum of average_math, average_reading, and average_writing, divided by a count of all schools, aliased as average_borough_sat    
    (SUM(average_writing) + SUM(average_reading) + SUM(average_math)) / COUNT(school_name) AS average_borough_sat
FROM schools    
-- Organize results by borough
GROUP BY borough
-- Display by average_borough_sat in descending order
ORDER BY average_borough_sat DESC;

 * sqlite:///createdb.db
Done.


borough,num_schools,average_borough_sat
Staten Island,10,1439
Queens,69,1345
Manhattan,89,1340
Brooklyn,109,1230
Bronx,98,1202


## 9. Brooklyn numbers
<p>It appears that schools in Staten Island, on average, produce higher scores across all three categories. However, there are only 10 schools in Staten Island, compared to an average of 91 schools in the other four boroughs!</p>
<p>For our final query of the database, let's focus on Brooklyn, which has 109 schools. We wish to find the top five schools for math performance.</p>

In [41]:
%%sql

-- Select school and average_math
SELECT 
    school_name,
    average_math
FROM schools    
-- Filter for schools in Brooklyn
WHERE borough IN ('Brooklyn')
-- Aggregate on school_name
GROUP BY school_name
-- Display results from highest average_math and restrict output to five rows
ORDER BY average_math DESC
LIMIT 5;

 * sqlite:///createdb.db
Done.


school_name,average_math
Brooklyn Technical High School,682
Brooklyn Latin School,625
Leon M. Goldstein High School for the Sciences,563
Millennium Brooklyn High School,553
Midwood High School,550
